Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 
 
# MONAI Label Hello, World! - Radiology App and Slicer Example

***The Active Learning Process with MONAI Label***

In this notebook, we provide a hello world example of MONAI Label use case. Using Spleen segmentation in Radiology app as the demonstration, 3D Slicer as the client viewer, we show how MONAI Label workflow serves as interacitve AI-Assisted tool for labeling CT scans. 

![workflow](./figures/monailabel_radiology_3dslicer/teaser_img.png)



## 1. Prepare MONAI Label

### Prerequisites
- **Install MONAI Label** weekly preview release: 


## Setup environment

In [ ]:
!pip install monailabel-weekly

### 1.1 Check Install and All Sample Apps

In [ ]:
!monailabel apps

### 1.2 Download radiology app
Choose "radiology" as the app

In [ ]:
!monailabel apps --download --name radiology --output apps

### 1.3 Download sample data
Download MSD Spleen dataset as the sample dataset using monailabel API. The task is the volumetric (3D) segmentation of the spleen from CT image. The segmentation of spleen is formulated as the voxel-wise 2-class classification. Each voxel is predicted as either foreground (spleen) or background. The dataset is from the 2018 MICCAI challenge.


In [ ]:
!monailabel datasets --download --name Task09_Spleen --output datasets

### 1.4 Starting MONAI Label Server

In [ ]:
!monailabel start_server --app apps/radiology --studies datasets/Task09_Spleen/imagesTr --conf models segmentation_spleen

## 2. Auto Segmentation with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for auto spleen segmentation. Pre-trained model is provided and loaded by MONAI Label server.

### Installation of 3D Slicer and MONAI Label Plugin

You can download 3D Slicer on the [3D Slicer download page](https://download.slicer.org/).
MONAI Label supports stable and preview versions of 3D Slicer: [Stable and Preview Version >= 5.0](https://download.slicer.org/).

**For Ubuntu**

**Prerequisites for 3D Slicer**: Slicer is expected to work on the vast majority of desktop and server Linux distributions. The system is required to provide at least GLIBC 2.17 and GLIBCCC 3.4.19. For more details, please refer to [prerequisites](https://slicer.readthedocs.io/en/latest/user_guide/getting_started.html#linux). 

As an example, Ubuntu users can click the downloaded 3D Slicer file and extract to the working folder, then click "Slicer" to open the 3D Slicer viewer. Specifically, Open the tar.gz archive and copy directory to the location of your choice. Installation of additional packages may be necessary depending on the Linux distribution and version. Run the Slicer executable to start the 3D Slicer.

**For Windows**

Download and run the 3D Slicer installer. Once installed, you can run `Slicer` from the Windows Start Menu.

#### Install MONAI Label plugin from Slicer Extension Manager

**Step 1:** Start 3D Slicer and navigate to **View** -> **Extension Manager** -> **Active Learning** -> **MONAI Label**

<img src="./figures/monailabel_radiology_3dslicer/3dslicer-extensions-manager.png" alt="drawing" width="600"/>

**Step 2:** Install the MONAI Label plugin (click "install") and restart 3D Slicer. The "Restart" button is in the lower right-hand corner of the Extension Manager dialog box.

**Step 3:** Add shortcut: Navigate to **Edit** -> **Application Settings**, under the **Modules** panel drag **MONAI Label** into the **Favorite Modules** panel. Click **OK**, restart 3D Slicer, and the MONAI Label icon <img src= "./figures/monailabel_radiology_3dslicer/MONAILabel.png" alt=" drawing" width=" 20"/> will be visible in the toolbar.

<img src="./figures/monailabel_radiology_3dslicer/monai-label-plugin-favorite-modules-1.png" alt="drawing" width="600"/>


### 2.1 Start 3D Slicer and MONAI Label Plugin

- On the menu bar navigate click MONAI Label, connect and Check MONAI Label Server Address/Name, radiology app, and confirm 3D Slicer is connected to MONAI Label server correctly, if no, fill in the correct server address and port.

![slicer1](./figures/monailabel_radiology_3dslicer/slicer1.png)


### 2.2 Select Model and Fetch Data

- Select the model users would like to use, such as "**segmentation_spleen**" in this tutorial, click **next sample** to load image data from study datastore.

![slicer1](./figures/monailabel_radiology_3dslicer/slicer2.png)


### 2.3 Run Auto Segmentation

- MONAI Label has provided pre-trained model weights for the spleen segmentation model, click **run** to do inference on the current loaded data. 

- Note: the pre-trained models will be downloaded when starting MONAI Label server in the "**model**" folder. For instance, in this use case, downloaded pre-trained model is saved at "**apps/radiology/model/pretrained_segmentation_spleen.pt**".

![slicer1](./figures/monailabel_radiology_3dslicer/slicer3.png)


### 2.4 Edit Annotations and Submit Label

- Users can edit the inference labels manually using Segment Editor, the tool for adjusting annotation. Once done annotation, click **Submit Label** to save the ground truth label to the file system.

- Note: the final annotations will be saved to "**labels/final**" folder in the study dataset, for example, in this use case, the ground truth label will be saved to "**datasets/Task09_Spleen/imagesTr/labels/final**"

#### AI assisted corrections:

![scribbles](./figures/monailabel_radiology_3dslicer/scribbles.png)

#### Corrections with Segment Editor:

![slicer1](./figures/monailabel_radiology_3dslicer/slicer4.png)


## 3. Active Learning with 3D Slicer

Active learning and interactive fine-tuning models have highlighted features in MONAI Label, users can train their models anytime when new annotated labels are saved. Click **Train** button, MONAI Label server will fetch saved final ground truth label and fine-tune the prior model. 

![slicer1](./figures/monailabel_radiology_3dslicer/slicer5.png)

Users can do auto segmentation using the latest fine-tuned model. 
Active learning process will select unlabeld batch data.

Next image selected, the already trained images will be marked as labeled data, then won't be selected for next labeling batch. Active learning strategies such as "first/random" will be used for selecting which unlabeled data to fetch. 

For general active learning workflow design, see the slide: ![activeL](./figures/monailabel_radiology_3dslicer/activeLearning.png)


## 4. Repeat the Interactive Labeling Process 

Repeated fetch data and active learning process to section 2.2, until all unlabeled data are annotated and trained. 

For each training loop, the new best metric model will be saved in "**model/<model_name>.pt**", in this use case, ""**apps/radiology/model/segmentation_spleen.pt**" is saved. For detailed training stats and step models, refer to "**apps/radiology/model/segmentation_spleen/train_01**", 

## 5. Conclusion

This tutorial demonstrates basic usage of MONAI Label with radiology app, a spleen segmentation task is introduced. Active learning process, auto segmentation, segmentation editor, submit labels, and save models are included. 
We highlight the interactive learning process when labeling a dataset, our platform can achieve incremental improvements and understand medical data better for researchers and clinicians. 
